In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
pretrained_model_path = r"D:\TensorflowPractice\venv\Models\Inception\inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [4]:
pretrained_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(150,150,3),
    include_top = False,
    weights = None
)

In [5]:
pretrained_model.load_weights(pretrained_model_path)

In [6]:
for layer in pretrained_model.layers:
    layer.trainable = False

In [7]:
last_layer = pretrained_model.get_layer('mixed7')
last_output = last_layer.output

In [8]:
last_output

<KerasTensor shape=(None, 7, 7, 768), dtype=float32, sparse=False, ragged=False, name=keras_tensor_228>

In [9]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024,activation='relu')(x)
x = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(pretrained_model.input,x)

model.compile(
    loss='binary_crossentropy', 
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), 
    metrics = ['accuracy'])

In [10]:
TRAIN_DIR = r"D:\TensorflowPractice\venv\Datasets\horse-or-human"
VAL_DIR = r"D:\TensorflowPractice\venv\Datasets\validation-horse-or-human"

train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size = (150,150),
    batch_size = 32,
    label_mode = 'binary'
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    image_size = (150,150),
    batch_size = 32,
    label_mode = 'binary'
)

Found 1027 files belonging to 2 classes.
Found 256 files belonging to 2 classes.


In [11]:
trained_dataset_shuffled = (
    train_dataset
    .cache()
    .shuffle(1000)
    .prefetch(tf.data.AUTOTUNE)
) 

validation_dataset_shuffled = (
    validation_dataset
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

In [12]:
rescale_layer = tf.keras.layers.Rescaling(1./255)

train_dataset_rescaled = trained_dataset_shuffled.map(lambda image, label: (rescale_layer(image),label))
validation_dataset_rescaled = validation_dataset_shuffled.map(lambda image, label: (rescale_layer(image),label))

In [13]:
model_augmentation = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(150,150,3)),
    tf.keras.layers.RandomFlip(mode="horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(factor=0.2,fill_mode='nearest'),
    tf.keras.layers.RandomTranslation(0.2,0.2,fill_mode='nearest'),
    tf.keras.layers.RandomZoom(0.2,fill_mode='nearest')
])

In [14]:
model_with_aug = tf.keras.Sequential([
    model_augmentation,
    model
])

In [15]:
model_with_aug.compile(loss='binary_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['accuracy'])

In [ ]:

history = model_with_aug.fit(
    train_dataset_rescaled,
    validation_data = validation_dataset_rescaled,
    epochs=20,
    verbose = 2
)

Epoch 1/20


KeyboardInterrupt: 